## Train a model 

In [83]:
%%bash
mlflow run bike-rental-regression -P train_year=2011 -P train_month=1 -P test_year=2011 -P test_month=3 -P dataset_path=train_bike.csv

2021/11/06 19:15:28 INFO mlflow.projects.utils: === Created directory /tmp/tmptcbb_mkk for downloading remote URIs passed to arguments of type 'path' ===
2021/11/06 19:15:28 INFO mlflow.projects.backend.local: === Running command 'source /home/andrea/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-3ba878fc7c575322867d4954b82239190f6a5ce6 1>&2 && python train.py 2011 1 2011 3 /home/andrea/demo-ATSS/train_bike.csv' in run with ID 'bd011c5a3b3944bd82d33d2319d1318c' === 
/home/andrea/miniconda3/envs/mlflow-3ba878fc7c575322867d4954b82239190f6a5ce6/lib/python3.9/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) t

## Serve a model

`mlflow models serve --model-uri runs:/bd011c5a3b3944bd82d33d2319d1318c/model --port 9999`

Do this in a separate shell as it will not terminate until we stop it.

Use the run ID from the previous step's ouput.

## Try to get a prediction

In [84]:
import requests
import json
import pandas as pd

In [85]:
dataset = pd.read_csv('train_bike.csv', header=0,  sep=',',
                      parse_dates=['datetime'], index_col='datetime').drop(
    columns=['casual', 'registered']
)
test_year = 2011
test_month = 3
test: pd.DataFrame = dataset.loc[
    lambda df: (df.index.year == test_year) & (df.index.month == test_month)
]
def time_features(input_df):
    return input_df.assign(
        month = lambda df: df.index.month,
        day = lambda df: df.index.day,
        hour = lambda df: df.index.hour,
    )
test.pipe(time_features).drop(columns='count').head().to_json(orient='split', index=False)

'{"columns":["season","holiday","workingday","weather","temp","atemp","humidity","windspeed","month","day","hour"],"data":[[1,0,1,1,12.3,13.635,70,31.0009,3,1,0],[1,0,1,1,10.66,11.365,70,23.9994,3,1,1],[1,0,1,1,9.84,10.605,65,26.0027,3,1,2],[1,0,1,1,9.02,10.605,69,19.0012,3,1,3],[1,0,1,1,9.02,10.605,69,16.9979,3,1,4]]}'

In [86]:
predictions = requests.post(
    'http://127.0.0.1:9999/invocations',
    json=json.loads(
        test.pipe(time_features)
        .drop(columns='count')
        .to_json(orient='split', index=False)
    )
).json()

In [87]:
predictions[:5]

[-2.6364862118705275,
 -1.7267545427918696,
 -6.801566608000301,
 -7.742135583613201,
 -7.333175706525842]

In [88]:
from sklearn.metrics import r2_score

In [89]:
r2_score(test.pipe(time_features)['count'].values, predictions)

0.6026563896049335